In [1]:
from pathlib import Path
import time

import essentia
import essentia.standard as es
import librosa
import librosa.display
# IMPORTANT: since TCN is not available on pip version of madmom
# you have to build the library from source
import madmom
import matplotlib.pyplot as plt
import mirdata
import mir_eval
import numpy as np
import pandas as pd

import src.baseline as baseline
import src.utils as utils

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [2]:
base_path = Path.cwd()

experiments_path = base_path / "results" / "baselines"
# output_path = base_path / "experiments_results" / "beat_trackers_baseline"
candombe_path = base_path.parent / "datasets" / "candombe"
sambaset_path = base_path.parent / "datasets" / "sambaset-mini"

In [3]:
print(experiments_path), print(candombe_path), print(sambaset_path)

/home/giovanamorais/Documents/personal/beat_detection_experiments/results/baselines
/home/giovanamorais/Documents/personal/datasets/candombe
/home/giovanamorais/Documents/personal/datasets/sambaset-mini


(None, None, None)

Important dataset information:
* Candombe:
    * sampling rate: 44100 Hz
    * precision: 16-bit
    * total audios: 36

In [10]:
gtzan = mirdata.initialize("gtzan_genre", data_home="/home/giovanamorais/Documents/personal/datasets/gtzan_genre", version="default")

In [6]:
SR = 44100

In [7]:
# helper dicts
baselines = {
    "librosa": baseline.librosa_beats,
    "madmom_rnn": baseline.madmom_rnn_beats,
    "madmom_tcn": baseline.madmom_tcn_beats,
    "essentia": baseline.essentia_beats
}

times = {
    "librosa": {},
    "madmom_rnn": {},
    "madmom_tcn": {},
    "essentia": {}
}

In [8]:
# candombe
audiofiles = candombe_path.rglob("*.flac")

for audio in audiofiles:
    x, _ = librosa.load(audio, mono=True, sr=SR)
    
    for key, val in baselines.items():
        file_npz = experiments_path / key / (audio.stem + ".npz")
        
        # if file exists, do nothing
        if not file_npz.is_file():
            start = time.perf_counter()
            beats = val(x)
            end = time.perf_counter()
            
            times[key][audio.stem] = end - start
            
            # is it better to create everything before looping?
            utils.create_folder(file_npz.parent)

            np.savez(file_npz, estimated=beats)

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-1.5258789e-05  1.5258789e-05 -1.5258789e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")


Creating folder for /home/giovanamorais/Documents/personal/beat_detection_experiments/results/baselines
Creating folder for /home/giovanamorais/Documents/personal/beat_detection_experiments/results/baselines
Creating folder for /home/giovanamorais/Documents/personal/beat_detection_experiments/results/baselines
Creating folder for /home/giovanamorais/Documents/personal/beat_detection_experiments/results/baselines


/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-1.5258789e-04 -2.1362305e-04 -1.5258789e-04 ...  1.5258789e-05
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[3.0517578e-05 1.5258789e-05 4.5776367e-05 ... 1.5258789e-05 1.5258789e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_d

In [11]:
# gtzan
for track_name in gtzan.track_ids:
    x, _ = librosa.load(gtzan.track(track_name).audio_path, mono=True, sr=SR)
    
    for key, val in baselines.items():
        file_npz = experiments_path / key / (track_name + ".npz")
                
        # if file exists, do nothing
        if not file_npz.is_file():
            start = time.perf_counter()
            beats = val(x)
            end = time.perf_counter()

            times[key][track_name] = end - start

            # is it better to create everything before looping?
            utils.create_folder(file_npz.parent)

            np.savez(file_npz, estimated=beats)

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.0198018  -0.02402364 -0.01773873 ...  0.03475156  0.03023224
  0.01556822] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.01115162 -0.01062855 -0.00237915 ... -0.04458708 -0.03788638
 -0.01936849] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.04016757 -0.0638983  -0.07841969 ... -0.03845555 -0.02856299
 -0.0136734 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats =

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.03744644 -0.05916126 -0.07250473 ... -0.00709576 -0.01008316
 -0.00615764] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.00950884 0.01531108 0.01868245 ... 0.05135197 0.04601406 0.02438169] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.07893597 -0.10387797 -0.09430623 ... -0.0687365  -0.05618517
 -0.02830622] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = libros

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.21672335 -0.23701766 -0.18243556 ...  0.07123011  0.04985653
  0.02276039] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.11320384 -0.15274504 -0.15486394 ... -0.06947306 -0.0599477
 -0.03152933] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.00284157 0.0024222  0.00157677 ... 0.0161042  0.01371586 0.00717695] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.02988874 0.03313692 0.02815039 ... 0.00891773 0.00669159 0.0033493 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.00567002 0.00480596 0.00271192 ... 0.02487803 0.02266176 0.01216435] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.02271146 0.02930846 0.02926713 ... 0.00934964 0.01256562 0.00772676] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_tr

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.0796275  -0.10593052 -0.09846676 ... -0.0626828  -0.05633827
 -0.02893967] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.02440504 -0.03886485 -0.04640672 ...  0.0521527   0.04363639
  0.02191063] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.00468852 0.00569011 0.00497851 ... 0.01570844 0.01301944 0.0065446 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = libros

...

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.04014984  0.04360585  0.03293891 ... -0.0145762  -0.01281489
 -0.00694241] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.01627747 -0.02184317 -0.01928948 ...  0.0880055   0.07509403
  0.03904391] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.33361688  0.38684544  0.3449403  ...  0.07747781  0.03237729
 -0.00796164] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats =

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 1.7004275e-01  1.4038613e-01  5.9871703e-02 ...  6.9672894e-03
 -7.1870112e-05 -5.1162317e-03] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.05505097 -0.04125232  0.00408849 ... -0.02198162 -0.04440418
 -0.03101136] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.12621255  0.1648235   0.16277803 ... -0.24172086 -0.18978491
 -0.09131828] as keyword args. From version 0.10 passing these as positional arguments will result in an er

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.03392928  0.05713617  0.07895762 ... -0.22395936 -0.18919468
 -0.09799828] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.11134558 -0.14046033 -0.13913517 ... -0.02698034 -0.0034723
  0.0028615 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.09206021 -0.09674683 -0.0623085  ... -0.434355   -0.33296004
 -0.16699708] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = 

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.08524905 -0.11453855 -0.10925654 ... -0.01328609 -0.01186775
 -0.00576854] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.132075    0.19166325  0.21052337 ... -0.07985376 -0.04542939
 -0.01834226] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.03354095 -0.03757039 -0.02274543 ...  0.04642467  0.01953384
  0.00482395] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats =

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.14269467 0.1719096  0.18772027 ... 0.08654857 0.07256123 0.03790819] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[-0.28570682 -0.3328773  -0.27956432 ... -0.11887106 -0.14314319
 -0.08629605] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.03887465 0.05458516 0.05614739 ... 0.024179   0.03453028 0.02322186] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.

In [12]:
# sambaset
annotations_path = sambaset_path / "annotations" / "beats"
audio_path = sambaset_path / "audio"

for i in annotations_path.rglob("*.beats"):
    folder = i.stem.split(".")[0]
    track_name = audio_path / folder / (i.stem + ".mp3")
    
    x, _ = librosa.load(track_name, mono=True, sr=SR)
    # skip first 30s
    x = x[30*SR::]
    for key, val in baselines.items():
        file_npz = experiments_path / key / (track_name.stem + ".npz")
                
        # if file exists, do nothing
        if not file_npz.is_file():
            start = time.perf_counter()
            beats = val(x)
            end = time.perf_counter()

            times[key][track_name.stem] = end - start

            # is it better to create everything before looping?
            utils.create_folder(file_npz.parent)

            np.savez(file_npz, estimated=beats)

/home/giovanamorais/Documents/personal/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[0.03938293 0.07122803 0.09759521 ... 0.         0.         0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[7.6293945e-05 3.0181885e-02 1.2207031e-04 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will

/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 1.5405273e-01  1.5028381e-01  1.5049744e-01 ... -1.5258789e-05
  0.0000000e+00  1.5258789e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/home/giovanamorais/Documents/personal/beat_detection_experiments/src/baseline.py:21: FutureWarning: Pass y=[ 0.02075195 -0.11975098  0.06552124 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bpm, beats = librosa.beat.beat_track(audio, sr=SR, units="time")
/home/giovanamorais/Documents/personal/venv/lib/python3.7/site-packages/librosa/util/decorators.py

# output examples 

datasets:

| track_id | dataset |
| --- | --- |
| track_1 | candombe | 
| track_2 | gtzan |


beats:

| track_id | dataset | reference | librosa_estimate | essentia_estimate | madmom_rnn_estimate | madmom_tcn_estimate |
| --- | --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | np.array([...]) | np.array([...]) | np.array([...]) | np.array([...]) | np.array([...]) | 

performance:

| track_id | dataset | librosa_time | essentia_time | madmom_rnn_time | madmom_tcn_time |
| --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | 0.99 | 0.98 | 0.87 | 0.99 | 1.2 | 


metrics:

| track_id | dataset | librosa_fmeasure | essentia_fmeasure | madmom_rnn_fmeasure | madmom_tcn_fmeasure |
| --- | --- | --- | --- | --- | --- | 
| track_1 | gtzan | 0.99 | 0.98 | 0.87 | 0.99 | 1.2 | 



In [ ]:
track_name.stem

In [13]:
# build "dataset" dataframe
datasets = {}

for i in gtzan.track_ids:
    # datasets[i] = f"gtzan.{i.split('.')[0]}"
    datasets[i] = "gtzan"


for i in candombe_path.rglob("*.flac"):
    datasets[i.stem] = "candombe"
    
for i in annotations_path.rglob("*.beats"):
    datasets[i.stem] = "sambaset-mini"

dataset_df = pd.DataFrame.from_dict(datasets, orient="index", columns=["dataset"])
dataset_df.index.name = "track_id"

dataset_df.to_csv(experiments_path / "experiment_data.csv")

In [14]:
dataset_df.dataset.unique()

array(['gtzan', 'candombe', 'sambaset-mini'], dtype=object)

In [15]:
# build time dataframe
times_df = pd.DataFrame(times)
times_df.index.name = "track_id"

In [16]:
times_df = times_df.reset_index()

In [17]:
import pathlib
times_df["track_id"] = times_df["track_id"].apply(lambda x: x.stem if type(x) == pathlib.PosixPath else x)

In [18]:
# times_df.to_csv(experiments_path / "processing_time.csv", index=False)

In [19]:
times_df = pd.read_csv(experiments_path / "processing_time.csv")
times_df = times_df.set_index("track_id")

In [20]:
times_df

,librosa,madmom_rnn,madmom_tcn,essentia
track_id,,,,
zavala.muniz.2014_55,4.318925,34.792489,73.462430,4.129101
csic.1995_ansina1_01,4.175075,37.558389,74.654578,3.391646
zavala.muniz.2014_53,2.866825,27.971717,56.853648,2.729417
csic.1995_ansina1_02,3.879108,38.920582,80.393221,3.751782
zavala.muniz.2014_49,2.060457,19.688643,41.787751,1.946116
...,...,...,...,...
VAS6.12,1.233866,12.360256,24.167959,1.715052
VAS3.04,2.176773,21.088533,45.262095,2.072672
VAS6.01,2.225423,22.476305,45.806524,2.135832


In [ ]:
genre_times = times_df.join(dataset_df)
genre_times

In [ ]:
genre_times.boxplot(by="dataset")

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
ax = times_df.join(dataset_df).groupby("dataset").boxplot(return_type="axes")
for i in ax.values:
    i.set_ylabel("segundos")
    i.set_xlabel("método")    

# plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# saving median time per dataset
# times_df.join(dataset_df).groupby("dataset").median().to_csv(experiments_path / "median_time_per_dataset.csv")

In [ ]:
dataset_df = pd.read_csv(experiments_path / "experiment_data.csv")
dataset_df = dataset_df.set_index("track_id")

In [ ]:
# TODO: document the default parameter for those algorithms
# TODO: add more baselines? (e.g librosa with multi-channel?)
# TODO: evaluate baselines against ground_truth value 
# TODO: parallelize experiments run 

In [ ]:
# we have to group information in a dataframe or anything like this to 
# better analyze

In [ ]:
# we're not worried about downbeat estimation, so let's first just save our beats

# candombe
csvfiles = candombe_path.rglob("*.csv")
reference = {}

for file in csvfiles:
    x_df = pd.read_csv(file, names=["timestamp", "beat"])
    
    reference[file.stem] = {}
    reference[file.stem]["reference"]  = x_df["timestamp"].values
    
# gtzan
for file in gtzan.track_ids:
    reference[file] = {}
    reference[file]["reference"] = gtzan.track(file).beats.times

# gather estimations
estimations = experiments_path.glob("*")
for folder in estimations:
    print(f"Reading files from /{folder.name}")
    for file in folder.glob("*.npz"):
        reference[file.stem][folder.name] = np.load(file)["estimated"]

In [ ]:
# isso aqui fica horroroso. melhor salvar em um arquivo h5 ou como pickle. o csv fica muito ruim.
beat_df = pd.DataFrame(reference).transpose()
beat_df.index.name = "track_id"
beat_df.to_csv(experiments_path / "estimation.csv")

In [ ]:
beat_df.info()

In [ ]:
beat_df = pd.read_csv(experiments_path / "estimation.csv")
beat_df = beat_df.set_index("track_id")

beat_df["reference"] = beat_df["reference"].to_numpy()
beat_df["madmom_rnn"] = beat_df["madmom_rnn"].to_numpy()
beat_df["madmom_tcn"] = beat_df["madmom_tcn"].to_numpy()
beat_df["librosa"] = beat_df["librosa"].to_numpy()
beat_df["essentia"] = beat_df["essentia"].to_numpy()

In [ ]:
beat_df.info()

In [ ]:
beat_df["madmom_rnn_metrics"] = beat_df[["reference", "madmom_rnn"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["madmom_rnn"]), axis=1)
beat_df["librosa_metrics"] = beat_df[["reference", "librosa"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["librosa"]), axis=1)
beat_df["madmom_tcn_metrics"] = beat_df[["reference", "madmom_tcn"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["madmom_tcn"]), axis=1)
beat_df["essentia_metrics"] = beat_df[["reference", "essentia"]].apply(lambda x: mir_eval.beat.evaluate(x["reference"], x["essentia"]), axis=1)

In [ ]:
def tmp(column_dict, index_column):
    column_dict["track_id"] = index_column
    return column_dict

In [ ]:
beat_df["madmom_rnn_metrics"].iloc[0]

In [ ]:
pd.json_normalize(beat_df["madmom_rnn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_rnn_metrics"], x["track_id"]), axis=1))

In [ ]:
# gambiarra nossa de cada dia
madmom_rnn_metrics = pd.json_normalize(beat_df["madmom_rnn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_rnn_metrics"], x["track_id"]), axis=1)).set_index("track_id")
madmom_tcn_metrics = pd.json_normalize(beat_df["madmom_tcn_metrics"].reset_index().apply(lambda x: tmp(x["madmom_tcn_metrics"], x["track_id"]), axis=1)).set_index("track_id")
librosa_metrics = pd.json_normalize(beat_df["librosa_metrics"].reset_index().apply(lambda x: tmp(x["librosa_metrics"], x["track_id"]), axis=1)).set_index("track_id")
essentia_metrics = pd.json_normalize(beat_df["essentia_metrics"].reset_index().apply(lambda x: tmp(x["essentia_metrics"], x["track_id"]), axis=1)).set_index("track_id")

In [ ]:
# saving metrics
madmom_tcn_metrics.to_csv(experiments_path / "madmom_tcn_metrics.csv")
madmom_rnn_metrics.to_csv(experiments_path / "madmom_rnn_metrics.csv")
librosa_metrics.to_csv(experiments_path / "librosa_metrics.csv")
essentia_metrics.to_csv(experiments_path / "essentia_metrics.csv")

In [ ]:
# saving median metric per dataset
# madmom_tcn_metrics.join(dataset_df).groupby("dataset").median().to_csv(experiments_path / "madmom_tcn_metrics_per_dataset.csv")
# madmom_rnn_metrics.join(dataset_df).groupby("dataset").median().to_csv(experiments_path / "madmom_rnn_metrics_per_dataset.csv")
# librosa_metrics.join(dataset_df).groupby("dataset").median().to_csv(experiments_path / "librosa_metrics_per_dataset.csv")
# essentia_metrics.join(dataset_df).groupby("dataset").median().to_csv(experiments_path / "essentia_metrics_per_dataset.csv")

In [ ]:
madmom_tcn_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
madmom_rnn_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
essentia_metrics.join(dataset_df).groupby("dataset").median()

In [ ]:
librosa_metrics.join(dataset_df).groupby("dataset").median()